# Projeto Health Insurance

## Desenvolvedora: Cinthya Oestreich Silva


# 1.0 Imports 

In [2]:

import sys 
import mysql.connector
import os
import psycopg2
import pandas as pd
import inflection 

# 2.0 Data Base - Dados

### O primeiro passo foi conhecer o banco de dados que nos foi fornecido. Tinhamos então um banco Postegress hosedado na AWS. Recebi as credenciais do banco e nada mais. 
### O primeiro desafio então foi descobrir quantas tabelas tinham no banco e o que continha dentro de cada tabela. 

### Nesta seção eu realizei o comando abaixo para recuperar o schema do banco e descobrir suas tabelas. 
### Na celular abaixo utilizei o psycop2 para realizar a query e receber as respostas. 

In [4]:
con = psycopg2.connect(host=host, database=database,
user=user, password=password)
cur = con.cursor()
sql = 'SELECT nspname from pg_catalog.pg_namespace pn'
cur.execute(sql)
recset = cur.fetchall()
for rec in recset:
    print(rec)
con.close()       

('pg_toast',)
('pg_temp_1',)
('pg_toast_temp_1',)
('pg_catalog',)
('information_schema',)
('public',)
('pa004',)


### Podemos ver que o Schema do Banco é que vamos utilizar é o pa004. 
### Com o schema em mãos podemos então descobrir quais tabelas estão dentro deste collection

In [5]:
con = psycopg2.connect(host=host, database=database,
user=user, password=password)
cur = con.cursor()
sql = "SELECT tablename from pg_catalog.pg_tables where schemaname = 'pa004' " 
cur.execute(sql)
recset = cur.fetchall()
for rec in recset:
    print(rec)
con.close()     

('users',)
('vehicle',)
('insurance',)


### Foi possivel observar que o banco que nos foi disponibilizado possui 3 tabelas users, Vehicle e insurance.
### Com esta informação vamos fazer uma consulta em cada tabela para visualizar os dados que pssuimos. 

In [6]:
con = psycopg2.connect(host=host, database=database,
user=user, password=password)
cur = con.cursor()
sql = "SELECT * FROM pa004.vehicle LIMIT 10"
cur.execute(sql)
recset = cur.fetchall()
for rec in recset:
    print(rec)
con.close()   

(1, 1, '> 2 Years', 'Yes')
(2, 1, '1-2 Year', 'No')
(3, 1, '> 2 Years', 'Yes')
(4, 1, '< 1 Year', 'No')
(5, 1, '< 1 Year', 'No')
(6, 1, '< 1 Year', 'Yes')
(7, 1, '< 1 Year', 'Yes')
(8, 1, '1-2 Year', 'Yes')
(9, 1, '< 1 Year', 'No')
(10, 1, '< 1 Year', 'No')


### Podemos ver que apesar de a querry retornar a resposta das linhas da tabelas não conseguimos observar o que significa cada coluna. Assim vou apresentar um metodo utilizando o pandas onde recebemos a querry com o titulos das colunas e a transformamos em dataframe. 

In [7]:
conn = psycopg2.connect(host=host, database=database,user=user, password=password)

In [8]:
querry="SELECT * FROM pa004.vehicle LIMIT 10"

In [9]:
df1= pd.read_sql(querry, conn)
df1.head()

,id,driving_license,vehicle_age,vehicle_damage
0,1,1,> 2 Years,Yes
1,2,1,1-2 Year,No
2,3,1,> 2 Years,Yes
3,4,1,< 1 Year,No
4,5,1,< 1 Year,No


### Legal! Agora conseguimos fazer um querry e já observar suas colunas. Mas e se eu for realizar varias vezes esse processo? Preciso sempre criar a conexão ? Bom, neste caso podemos fazer uma função.
### A função que será criada recebe uma query sql e retorna um dataframe. 

In [10]:
def query_toDF(query):
    conn = psycopg2.connect(host=host, database=database,user=user, password=password)
    df1=pd.read_sql(query, conn)
    conn.close()
    return df1
    

In [11]:
df_users = query_toDF("SELECT * FROM pa004.users LIMIT 10")
df_vehicle = query_toDF("SELECT * FROM pa004.vehicle LIMIT 10")
df_insurance = query_toDF("SELECT * FROM pa004.insurance LIMIT 10")

In [12]:
df_users.head()

,id,gender,age,region_code,policy_sales_channel
0,1,Male,44,28.0,26.0
1,2,Male,76,3.0,26.0
2,3,Male,47,28.0,26.0
3,4,Male,21,11.0,152.0
4,5,Female,29,41.0,152.0


In [13]:
df_vehicle.head()

,id,driving_license,vehicle_age,vehicle_damage
0,1,1,> 2 Years,Yes
1,2,1,1-2 Year,No
2,3,1,> 2 Years,Yes
3,4,1,< 1 Year,No
4,5,1,< 1 Year,No


In [14]:
df_insurance.head()

,id,previously_insured,annual_premium,vintage,response
0,1,0,40454.0,217,1
1,2,0,33536.0,183,0
2,3,0,38294.0,27,1
3,4,1,28619.0,203,0
4,5,1,27496.0,39,0


### Conseguimos verificar todos as tabelas e seus atributos, porém agora é necessário realizar um join nas tabelas para que seja possivel começarmos a estudar sobres estes atributos. 
### Vou então fazer um join pelo ID. Lembrando que neste caso a pessoa que desenvolveu o banco nos contou que cada linha era um entrada de uma pessoa. 

In [15]:
df_inicial=query_toDF("select * from pa004.users u inner join pa004.vehicle v on (u.id=v.id) inner join pa004.insurance i on (i.id = u.id) ")
df_inicial.head()

,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,7,Male,23,11.0,152.0,7,1,< 1 Year,Yes,7,0,23367.0,249,0
1,13,Female,41,15.0,14.0,13,1,1-2 Year,No,13,1,31409.0,221,0
2,18,Female,25,35.0,152.0,18,1,< 1 Year,No,18,1,46622.0,299,0
3,31,Female,26,8.0,160.0,31,1,< 1 Year,No,31,0,2630.0,136,0
4,39,Male,45,8.0,124.0,39,1,1-2 Year,Yes,39,0,42297.0,264,0


### Muito bem agora temos nosso dataframe com todos os atributos. 
### O primeiro passo é deletar as colunas que não tem muito valor, e neste caso são as colunas de ID. 

In [16]:
df_drop=df_inicial.drop(columns=['id'])
df_drop.head()

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Male,23,11.0,152.0,1,< 1 Year,Yes,0,23367.0,249,0
1,Female,41,15.0,14.0,1,1-2 Year,No,1,31409.0,221,0
2,Female,25,35.0,152.0,1,< 1 Year,No,1,46622.0,299,0
3,Female,26,8.0,160.0,1,< 1 Year,No,0,2630.0,136,0
4,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0


### Neste caso como tinhamos algumas colunas com o titulo de id foram removidas. 

O conjunto de dados inclui as seguintes informações:

* Id: identificador único do cliente.
* Gender: gênero do cliente.
* Age: idade do cliente.
* Driving License: 0, o cliente não tem permissão para dirigir e 1, o cliente tem para dirigir ( CNH – Carteira Nacional de Habilitação )
* Region Code: código da região do cliente.
* Previously Insured: 0, o cliente não tem seguro de automóvel e 1, o cliente já tem seguro de automóvel.
* Vehicle Age: idade do veículo.
* Vehicle Damage: 0, cliente nunca teve seu veículo danificado no passado e 1, cliente já teve seu veículo danificado no passado.
* Anual Premium: quantidade que o cliente pagou à empresa pelo seguro de saúde anual.
* Policy sales channel: código anônimo para o canal de contato com o cliente.
* Vintage: número de dias que o cliente se associou à empresa através da compra do seguro de saúde.
* Response: 0, o cliente não tem interesse e 1, o cliente tem interesse.

# 3. Descrição dos dados

In [17]:
df1=df_drop

Podemos ver que as colunas estão como snakecase então vamos deixa-las do jeito que está. Caso estivessem concatenadas eu iria aplicar um função para deixa-las ajustada de acordo com o padrão snakecase. 

In [18]:
df1.columns

Index(['gender', 'age', 'region_code', 'policy_sales_channel',
       'driving_license', 'vehicle_age', 'vehicle_damage',
       'previously_insured', 'annual_premium', 'vintage', 'response'],
      dtype='object')

## 3.1 Data Dimesion - Dimensão dos Dados

Nest tópico vamos verificar quantas colunas e quantas linhas o dataframe possui. 
Neste caso possuimos 381109 linhas e 111 colunas. 

In [19]:
print("Number of Rows:{}" .format(df1.shape[0]))
print("Number of Columns:{}" .format(df1.shape[1]))

Number of Rows:381109
Number of Columns:11


## 3.2 Data Types - Tipo dos dados
Neste tópico vamos verificar os ripos das colunas e modifica-las. 
Neste caso podemos observar que as colunas que possuem números estão de acordo com o tipo, porém as colunas que são string não estáo de acordo com seu tipo. Vamos então modifica-las. 

In [20]:
df1.dtypes

gender                   object
age                       int64
region_code             float64
policy_sales_channel    float64
driving_license           int64
vehicle_age              object
vehicle_damage           object
previously_insured        int64
annual_premium          float64
vintage                   int64
response                  int64
dtype: object

In [21]:
#df1[.loc[1:381109]=df1['gender'].astype(str)
#df1['gender'].astype(str)
df1.dtypes

SyntaxError: invalid syntax (<ipython-input-21-c67ba850d415>, line 1)

## 3.3 Check NA - Verificar NA 
Neste tópico vamos verificar se existe algum campo vazio. Neste caso não existem campos vazios. 

In [22]:
df1.isna().sum()

gender                  0
age                     0
region_code             0
policy_sales_channel    0
driving_license         0
vehicle_age             0
vehicle_damage          0
previously_insured      0
annual_premium          0
vintage                 0
response                0
dtype: int64